# Quench Spectral function

Here we consider a novel way to obtain the spectral properties of the physical system following https://arxiv.org/pdf/1910.02839. 

- Typically, we can obtain the spectral properties of the physical systems through the **double Fourier transformation** of the **unequal-time correlators** (UTC) from an **equilibrium state**, known as the spectral function or the dynamical structure factor. This is closely related to the pump-prove spectroscopic techniques in the experiements.

- The newly introduct approach extracts the similar information from  the **double Fourier transformation** of the **equal-time correlators** (ETC) from a **non-equilibirum state**.



----

In general the unequal-time correlators are given as

<img src="./img/1.png" width="500" /> 

and the euqal-time correlators are given as

<img src="./img/2.png" width="400" />.


By inserting the complete basis using the Hamiltonian eignestates, the double Fourier transformation of the ETC has the form

<img src="./img/3.png" width="500" />.



----

Now suppose the non-equilibrium state we consider is a pure state that has large overlap with the ground state, i.e.,

<img src="./img/4.png" width="500" />.


One example is 
$$
| \psi \rangle = (1 - \epsilon) | 0 \rangle + \sum_{n\neq0} \epsilon_n |n\rangle, \qquad \epsilon_n << 1
$$

Then the above Fourier transformation of the ETC can be expanded to 

<img src="./img/5.png" width="700" />,

where the first term is just a delta peak at $\omega=0$ and the last term gives the non-trivial contribution for positive frequency.

Starting from the ground state $|0\rangle$, we see the intermediate state has momentum $P_m = k$ and the excitation state $| n \rangle $ has to have zero momentum indicating the second excitation from $m$ has to have opposite momentum $-k$, i.e., $|n\rangle = \hat{b}^\dagger_{-k}\hat{b}^\dagger_{k}|0\rangle$ with energy $E_n = 2E_k$.

By this arugment we expect to obtain,

<img src="./img/6.png" width="300" />

where the weight $F(k)$ depends on the operators $\hat{O}_1$ and $\hat{O}_2$. The form indicate that we can also probe the excitation spectrum through the Fourier transformation of the ETC from a non-equilbrium state.

------

## Example: Transverse field Ising model

An example is given by https://scipost.org/SciPostPhys.14.6.151 using Gaussian simulation. Aside from the contraint in the ansatz, the work exactly follows the protocal above to compute the quench spectral function.

The Hamiltonian is given as
$$
H = - \sum_{\langle i, j \rangle} X_i X_j - \sum_i g Z_i
$$

Starting from infinite field ground state $g\rightarrow\infty$, they consider the quench to finite $g$ at the paramagnetic phase, i.e., not crossing the phase transition, and show that the can obtain the spectral function correctly.


<img src="./img/QSF.png" width="700" />


### Below we try to reproduce the same result.